# Домашнее задание. Классификация изображений

Сегодня вам предстоить помочь телекомпании FOX в обработке их контента. Как вы знаете, сериал "Симпсоны" идет на телеэкранах более 25 лет, и за это время скопилось очень много видеоматериала. Персоонажи менялись вместе с изменяющимися графическими технологиями, и Гомер Симпсон-2018 не очень похож на Гомера Симпсона-1989. В этом задании вам необходимо классифицировать персонажей, проживающих в Спрингфилде. Думаю, нет смысла представлять каждого из них в отдельности.



In [5]:
!zip -r checkpoints.zip checkpoints

  adding: checkpoints/ (stored 0%)
  adding: checkpoints/epoch=24-val_f1=0.872.ckpt (deflated 8%)


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%env NVIDIA_VISIBLE_DEVICES=1
%env CUDA_VISIBLE_DEVICES=0

!echo $NVIDIA_VISIBLE_DEVICES
!echo $CUDA_VISIBLE_DEVICES

env: NVIDIA_VISIBLE_DEVICES=1
env: CUDA_VISIBLE_DEVICES=0
1
0


### Установка зависимостей

In [3]:
# we will verify that GPU is enabled for this notebook
# following should print: CUDA is available!  Training on GPU ...
# 
# if it prints otherwise, then you need to enable GPU: 
# from Menu > Runtime > Change Runtime Type > Hardware Accelerator > GPU

import torch
import numpy as np

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


В нашем тесте будет 990 картнок, для которых вам будет необходимо предсказать класс.

In [4]:
import pickle
import numpy as np
from skimage import io

from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path

from torchvision import transforms as TT
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader, Subset
import torch.nn as nn

from matplotlib import colors, pyplot as plt
%matplotlib inline

# в sklearn не все гладко, чтобы в colab удобно выводить картинки 
# мы будем игнорировать warnings
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)


In [5]:
# разные режимы датасета 
DATA_MODES = ['train', 'val', 'test']
# все изображения будут масштабированы к размеру 224x224 px
RESCALE_SIZE = 224
# работаем на видеокарте
DEVICE = torch.device("cuda")

https://jhui.github.io/2018/02/09/PyTorch-Data-loading-preprocess_torchvision/


Ниже мы исспользуем враппер над датасетом для удобной работы. Вам стоит понимать, что происходит с LabelEncoder и  с torch.Transformation. 

ToTensor конвертирует  PIL Image с параметрами в диапазоне [0, 255] (как все пиксели) в FloatTensor размера (C x H x W) [0,1] , затем производится масштабирование:
$input = \frac{input - \mu}{\text{standard deviation}} $, <br>       константы - средние и дисперсии по каналам на основе ImageNet


Стоит также отметить, что мы переопределяем метод __getitem__ для удобства работы с данной структурой данных.
 Также используется LabelEncoder для преобразования строковых меток классов в id и обратно. В описании датасета указано, что картинки разного размера, так как брались напрямую с видео, поэтому следуем привести их к одному размер (это делает метод  _prepare_sample) 

In [6]:
def imshow(inp, title=None, plt_ax=plt, default=False):
    """Imshow для тензоров"""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt_ax.imshow(inp)
    if title is not None:
        plt_ax.set_title(title)
    plt_ax.grid(False)

def visualize(dataset, labels=None):

    fig, ax = plt.subplots(nrows=3, ncols=3,figsize=(8, 8), \
                        sharey=True, sharex=True)
    for fig_x in ax.flatten():
        random_characters = int(np.random.uniform(0,len(dataset)))
        
        if labels is not None:
            im_val, label = dataset[random_characters]
            img_label = " ".join(map(lambda x: x.capitalize(), \
                                     dataset.dataset.classes[label].split('_')))
            imshow(im_val, title=img_label, plt_ax=fig_x)
            
        else:
            im_val = dataset[random_characters]
            imshow(im_val, plt_ax=fig_x)
            
        fig_x.axis('off')

In [7]:
class TestDataset(Dataset):
    def __init__(self, files, transform=None):
        super().__init__()
        
        self.transform = transform
        self.files = sorted(files)

        self.len_ = len(self.files)
        
    def load_sample(self, file):
        image = Image.open(file)
        image.load()
        return image
  
    def __getitem__(self, index):
        x = self.load_sample(self.files[index])
        
        x = self.transform(x)
        
        return x
    
    def __len__(self):
        return self.len_

In [8]:
from torchvision import datasets
from sklearn.model_selection import train_test_split
from torch.utils.data import WeightedRandomSampler
from copy import copy # bad

def get_dataloader(kind=None, batch_size=64):
    
    if kind is None:
        raise Exception
    
    transforms = {
        'default': TT.Compose([
            TT.Resize((224, 224)),
            TT.ToTensor(),
            TT.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ]),
        
        'train': TT.Compose([
            TT.RandomHorizontalFlip(p=0.7),
            TT.RandomResizedCrop(224 ,scale=(0.5, 1)),  # scale - min; max area of crop
            TT.RandomRotation(25),
            TT.GaussianBlur(9), 
            TT.ToTensor(),
            TT.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])
    }   
    
    if kind == 'train':
        
        dataset = datasets.ImageFolder(root=kind + '/', transform=transforms['default'])
        
        targets = np.array(dataset.targets, dtype=np.int)
        
        train_idx, valid_idx = train_test_split(
            np.arange(len(targets)),
            test_size=0.2,
            shuffle=True,
            stratify=targets
        )
        
        train_idx = np.array(train_idx, dtype=np.int)
        valid_idx = np.array(valid_idx, dtype=np.int)
        
        _, class_cnts = np.unique(targets[train_idx], return_counts=True)
        
        sample_weights = torch.FloatTensor(len(targets[train_idx]))
        
        for i, label_id in enumerate(train_idx):
            sample_weights[i] = 1. / class_cnts[targets[label_id]]
            
        
        train_subset = Subset(dataset, train_idx)
        valid_subset = Subset(dataset, valid_idx)
        
        train_subset.dataset = copy(dataset) # bad
        train_subset.dataset.transform = transforms['train'] # bad
        
        N = int(max(class_cnts) * len(class_cnts))
        
        train_sampler = WeightedRandomSampler(sample_weights, num_samples=N//2)
        
        train_loader = DataLoader(
            train_subset,
            batch_size=batch_size, 
            sampler=train_sampler,
            pin_memory=True,
            num_workers=40
        )
        
        val_loader = DataLoader(
            valid_subset,
            batch_size=batch_size, 
            pin_memory=True,
            num_workers=40
        )
    
        return train_loader, val_loader
    
    else:
        test_files = sorted(list(Path('test/').rglob('*.jpg')))
        
        dataset = TestDataset(test_files, transform=transforms['default'])
        
        return DataLoader(dataset, batch_size=64, num_workers=40)

In [9]:
BATCH_SIZE=256

In [36]:
train_loader, val_loader = get_dataloader('train', batch_size=BATCH_SIZE)

In [10]:
test_loader = get_dataloader('test', batch_size=BATCH_SIZE)

Давайте посмотрим на наших героев внутри датасета.

In [13]:
# visualize(test_loader.dataset)

In [16]:
# visualize(train_loader.dataset, labels=True)

In [18]:
# visualize(val_loader.dataset, labels=True)

In [20]:
# def predict(model, test_loader):
#     with torch.no_grad():
#         logits = []
    
#         for inputs in test_loader:
#             inputs = inputs.to(DEVICE)
#             model.eval()
#             outputs = model(inputs).cpu()
#             logits.append(outputs)
            
#     probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
#     return probs

In [11]:
from models import ClassificationNet

In [12]:
# ClassificationNet?

In [13]:
N_EPOCHS=30

In [14]:
import gc

torch.cuda.empty_cache()
gc.collect()

0

In [25]:
model = ClassificationNet(
    num_classes=42,
    freeze_ratio=0.7,
    n_epochs=N_EPOCHS
).to(DEVICE)

In [26]:
import wandb

In [27]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

wb_logger = pl.loggers.WandbLogger(
    name=f"Pretrain_ResNet50|bs={BATCH_SIZE}|n_epochs={N_EPOCHS}|aug|freeze={0.7}_macro_f1_balanced_train|reg-",
    project='simpsons'
)

checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints',
    monitor='val_f1',
    filename='{epoch:02d}-{val_f1:.3f}',
    mode='max'
)

trainer = pl.Trainer(
    max_epochs=N_EPOCHS,
    logger=wb_logger,
    accelerator='gpu',
    devices=1,
    benchmark=True,
    callbacks=[checkpoint_callback]
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: scalyvladimir. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [28]:
trainer.fit(
    model = model,
    train_dataloaders = train_loader,
    val_dataloaders = val_loader
)

wandb.finish()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name     | Type    | Params
-------------------------------------
0 | model    | ResNet  | 23.6 M
1 | sm       | Softmax | 0     
2 | train_f1 | F1Score | 0     
3 | val_f1   | F1Score | 0     
-------------------------------------
17.0 M    Trainable params
6.6 M     Non-trainable params
23.6 M    Total params
94.376    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=30` reached.


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train_f1,▁▂▃▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇██▇▇█▇▇▇██▇▇▇▇█
train_loss,█▆▆▅▄▄▄▄▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▁▂▂▂▁▁▂▂▂▂▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_f1,▁▄▄▅▆▆▆▆▆▇▇▇▇▇████████████████
val_loss,█▅▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,29
train_f1,0.92717
train_loss,2.82828
trainer/global_step,4439
val_f1,0.86592


In [529]:
wandb.finish()

In [17]:
ls checkpoints

'epoch=24-val_f1=0.872.ckpt'


In [29]:
model.load_state_dict(torch.load('checkpoints/epoch=24-val_f1=0.872.ckpt')['state_dict'])

<All keys matched successfully>

In [26]:
model = ClassificationNet(num_classes=42, freeze_ratio=0.7, n_epochs=N_EPOCHS)

In [30]:
def predict_test(model, data):
    predictions = torch.cat(trainer.predict(model, data)).numpy()
    
    return [train_loader.dataset.dataset.classes[x] for x in predictions]

In [34]:
# model.eval()

# preds = []

# for x in test_loader:
#     preds += torch.argmax(model.sm(model(x)), dim=1)
# preds

In [37]:
res = [train_loader.dataset.dataset.classes[x] for x in preds]

### Ну и что теперь со всем этим делать?

![alt text](https://www.indiewire.com/wp-content/uploads/2014/08/the-simpsons.jpg)

### Submit на Kaggle

![alt text](https://i.redd.it/nuaphfioz0211.jpg)

In [39]:
! ls 

checkpoints  __pycache__	    simpsons_kaggle_classifier.ipynb  train
models.py    sample_submission.csv  test			      wandb


In [39]:
test_filenames = [path.name for path in sorted(list(Path('test/').rglob('*.jpg')))]

In [40]:
import pandas as pd
my_submit = pd.read_csv("sample_submission.csv")
my_submit = pd.DataFrame({'Id': test_filenames, 'Expected': res})
my_submit.head()

,Id,Expected
0,img0.jpg,nelson_muntz
1,img1.jpg,bart_simpson
2,img10.jpg,ned_flanders
3,img100.jpg,chief_wiggum
4,img101.jpg,apu_nahasapeemapetilon


In [ ]:
# TODO : сделайте сабмит (это важно, если Вы не справляетесь, но дошли до этой ячейки, то сообщите в чат и Вам помогут)

In [41]:
my_submit.to_csv('sub1.csv', index=False)

## Приключение?

А теперь самое интересное, мы сделали простенькую сверточную сеть и смогли отправить сабмит, но получившийся скор нас явно не устраивает. Надо с этим что-то сделать. 

Несколько срочныйх улучшейни для нашей сети, которые наверняка пришли Вам в голову: 


*   Учим дольше и изменяем гиперпараметры сети
*  learning rate, batch size, нормализация картинки и вот это всё
*   Кто же так строит нейронные сети? А где пулинги и батч нормы? Надо добавлять
*  Ну разве Адам наше все? [adamW](https://www.fast.ai/2018/07/02/adam-weight-decay/) для практика, [статейка для любителей](https://openreview.net/pdf?id=ryQu7f-RZ) (очень хороший анализ), [наши ](https://github.com/MichaelKonobeev/adashift/) эксперименты для заинтересованных.

* Ну разве это deep learning? Вот ResNet и Inception, которые можно зафайнтьюнить под наши данные, вот это я понимаю (можно и обучить в колабе, а можно и [готовые](https://github.com/Cadene/pretrained-models.pytorch) скачать).

* Данных не очень много, можно их аугументировать и  доучититься на новом датасете ( который уже будет состоять из, как  пример аугументации, перевернутых изображений)

* Стоит подумать об ансамблях


Надеюсь, что у Вас получится!

![alt text](https://pbs.twimg.com/profile_images/798904974986113024/adcQiVdV.jpg)
